In [1]:
import requests
import json
import pandas as pd
from jsonpath_ng import jsonpath,parse
import re
import html2text
import os
import requests
from urllib.parse import urlparse

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
def extract_json(json_data:list):
    law_list = []
    ann_list = []
    for row in json_data:
        if 'rows' in row.keys():
            latest_update_found = False
            law_count = 0
            latest_law_data = None
            
            for law_data in row['rows']:
                
                #if no rows in keys then it's law record, if rows in keys it's folder not record
                #check latest update law 'ฉบับ update ล่าสุด' in nm or 'UPDATE' in id
                if 'rows' not in law_data.keys():
                    law_count += 1
                    if 'ฉบับ update ล่าสุด' in h.handle(law_data['data'][0]['value']) or 'UPDATE' in law_data['id']:
                        latest_update_found = True
                        latest_law_data = law_data
                        break
            #if no 'ฉบับ update ล่าสุด' in nm and 'UPDATE' in id and only1 record in main folder, it's mean that is latest law
            if not latest_update_found and law_count == 1:
                #get the first one law in main folder
                latest_law_data = next(law_data for law_data in row['rows'] if 'rows' not in law_data.keys())
    
            
            for law_data in row['rows']:
                #law filter
                if 'rows' not in law_data.keys():
                    law_d = {}
                    law_d['main_id'] = row['id']
                    law_d['name'] = row['data'][0]['value']
                    law_d['law_id'] = law_data['id']
                    law_d['law_nm'] = h.handle(law_data['data'][0]['value'])
                    law_d['law_link'] = law_data['data'][1]
                    law_d['latest_update'] = (law_data == latest_law_data)
                    law_list.append(law_d)
                else:
                #folder
                    for ann_data in law_data['rows']:
                        ann_d = {}
                        ann_d['main_id'] = row['id']
                        ann_d['name'] = row['data'][0]['value']
                        ann_d['announce_id'] = law_data['id']
                        ann_d['announce_type_nm'] = h.handle(law_data['data'][0]['value'])
                        ann_d['announce_sub_id'] = ann_data['id']
                        ann_d['announce_name'] = h.handle(ann_data['data'][0]['value'])
                        ann_d['announce_link'] = ann_data['data'][1]
                        ann_d['latest_update'] = False  # Announcements are not considered for latest update
                        ann_list.append(ann_d)
    return law_list , ann_list
# law_df = pd.DataFrame(law_list)
# ann_df = pd.DataFrame(ann_list)

In [3]:

#Apply Function for extract law_link column in law_df.
def extract_links(html):
    links = {}
    pattern = r"href='(https://krisdika\.ocs\.go\.th/librarian/get\?sysid=\d+&ext=(\w+))'"
    matches = re.findall(pattern, html)
    
    for url, ext in matches:
        if ext == 'pdf':
            if 'icon_en' in html and url == matches[-1][0]:
                links['en_pdf'] = url
            else:
                links['pdf'] = url
        elif ext == 'htm':
            links['html'] = url
        elif ext == 'tif':
            links['tif'] = url
    base_url = matches[0][0].rsplit('ext=', 1)[0]
    links['doc'] = base_url + 'ext=doc'
       
    
    return links

# law_df =law_df.join(law_df['law_link'].apply(extract_links).apply(pd.Series))




In [5]:
#download for row
def download_file(url):
    folder = r'/Users/nachanon/Documents/law_extract/files'
    os.makedirs(folder , exist_ok = True)
    #check url is not nan
    if pd.isna(url):
        return None

    #get path , filename 
    filename = str(url).split('=')[1].split('&')[0]
    ext = url.split('=')[2]
    full_name = filename + '.' + ext
    file_path = os.path.join(folder, full_name)

    #check url if there is a htm file should fill getfile in url to get content
    if ext == 'htm' or ext == 'html':
        if 'getfile' not in url:
            url = url.replace('get','getfile')
    response = requests.get(url , stream = True , verify = False )  
    if response.status_code == 200:
        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Downloaded: {file_path}")

        if os.path.getsize(file_path) == 0:
            print(f"Warning: Downloaded file is empty: {file_path}")
            os.remove(file_path)
            return None
        
        print(f"Completely downloaded: {file_path}")
        return file_path
    else:
        print(f'Fail to downloaded: {file_path}')
        return None
#encap for all file extension cols
def download_all_file(row):
    ext_cols = ['pdf','html','doc','tif','en_pdf']
    for ext in ext_cols:
        if ext in row.index:
            row[f'{ext}_path'] = download_file(row[ext])
        else:
            print(f'{ext} not found in this row!!')
    return row
# law_df = law_df.apply(download_all_file, axis =1)


In [ ]:
file_store = r"\\wsl.localhost\Ubuntu-20.04\home\tete\lawpdf_scraping3\files"
thai_consonants = [chr(i) for i in range(0x0E01, 0x0E2E + 1) if i not in [0x0E24, 0x0E26]]

h = html2text.HTML2Text()
h.ignore_links = True

for alpha in thai_consonants:
    alpha_url = f"https://krisdika.ocs.go.th/th/web/guest/law?p_p_id=LawPortlet_INSTANCE_aAN7C2U5hENi&p_p_lifecycle=2&p_p_state=normal&p_p_mode=view&p_p_cacheability=cacheLevelPage&p_p_lifecycle=1&_LawPortlet_INSTANCE_aAN7C2U5hENi_character={alpha}&_LawPortlet_INSTANCE_aAN7C2U5hENi_javax.portlet.action=selectCharacterMenu&_LawPortlet_INSTANCE_aAN7C2U5hENi_groupOfAct=byCharacter&_LawPortlet_INSTANCE_aAN7C2U5hENi_lawTypeId=2"
    alpha_response = requests.request("GET",alpha_url, verify=False)
    alpha_json = json.loads(alpha_response.text)['rows']
    
    law_list,ann_list = extract_json(alpha_json)
    law_df = pd.DataFrame(law_list)
    ann_df = pd.DataFrame(ann_list)

    law_df =law_df.join(law_df['law_link'].apply(extract_links).apply(pd.Series))
    law_df = law_df.apply(download_all_file, axis =1)